# Yield Curve Construction Using the Bootstrap Method

## Overview

In the previous notebook, we worked with a `ZeroCurve` class where we manually added zero rates at specific maturities. In practice, however, zero rates aren't directly observable in the market. Instead, we observe prices of traded instruments like bonds and bills, and we need to **extract** or **imply** the zero rates from these prices.

This notebook introduces the `YieldCurve` class, which extends the `ZeroCurve` class with a powerful method called **bootstrapping** that constructs a zero curve from a portfolio of market instruments.

## The YieldCurve Class: Inheritance in Action

The `YieldCurve` class demonstrates **inheritance** - another fundamental OOP principle:

```python
class YieldCurve(ZeroCurve):
    def __init__(self):
        super().__init__()
        self.portfolio = []
```

**What's happening here:**
- `YieldCurve` **inherits from** `ZeroCurve` (indicated by `class YieldCurve(ZeroCurve):`)
- This means YieldCurve automatically gets all the methods and attributes from ZeroCurve
- It can use `add_zero_rate()`, `get_discount_factor()`, `npv()`, etc. without rewriting them
- It adds new functionality (the `bootstrap()` method) while keeping all the base functionality
- The `super().__init__()` call ensures the parent class initialization runs first

**Why use inheritance?**
- **Code reuse**: We don't duplicate the ZeroCurve functionality
- **Extension**: We add specialized methods for yield curve construction
- **Is-a relationship**: A YieldCurve "is a" ZeroCurve (with extra capabilities)

## The Bootstrap Method: How It Works

The bootstrap method extracts zero rates from a portfolio of market instruments by working through them sequentially, from shortest to longest maturity.

**Algorithm Steps:**

1. **Initialize**: Add a zero rate of 0% at time 0 (present day reference point)

2. **Process Bank Bills**: 
   - Bank bills are simple instruments with one cash flow (face value at maturity)
   - Discount factor = Price / Face Value
   - This directly gives us the discount factor at each bill's maturity
   - Example: A 3-month bill trading at $98.75 with $100 face value → discount factor = 0.9875

3. **Process Bonds** (the clever part):
   - Bonds have multiple cash flows (coupons + principal)
   - We process them in maturity order, shortest to longest
   - For each bond:
     - Calculate the PV of all **known** cash flows (those with maturities already in our curve)
     - Subtract this PV from the bond's price to get the PV of the final cash flow
     - Solve for the discount factor at the bond's maturity:
     
     $$\text{DF}(T) = \frac{\text{Bond Price} - \text{PV of known cash flows}}{\text{Final cash flow amount}}$$

**Key Requirement**: The portfolio must be structured so that each bond introduces only **one new maturity point** (its final maturity). All intermediate coupon dates must already be covered by earlier instruments.

**Example:**
- Bill at 0.25 years → gives us DF(0.25)
- Bill at 0.5 years → gives us DF(0.5)
- Bond maturing at 1 year with semi-annual coupons → coupons at 0.5 and 1.0 years
  - We already know DF(0.5) from the bill
  - We can solve for DF(1.0) using the bond price

This sequential process "bootstraps" our way up the yield curve, building knowledge at each step!

## Import Required Libraries

Let's start by importing the necessary modules and libraries.

In [3]:
import importlib
import curve_classes_and_functions as yCurve
importlib.reload(yCurve)
import instrument_classes as inst
importlib.reload(inst)
import pandas as pd
import math

## Practical Example: Building a Yield Curve from Market Instruments

Now let's see the bootstrap method in action. We'll:
1. Create a portfolio of bank bills and bonds with market prices
2. Instantiate a YieldCurve object (which inherits all ZeroCurve functionality)
3. Use the bootstrap method to extract the zero curve from instrument prices

In [4]:


# create a portfolio of two bank_bills
bank_bill_1 = inst.Bank_bill(face_value=100, maturity=.25)
bank_bill_1.set_ytm(.05)
bank_bill_1.set_cash_flows()
print(bank_bill_1.get_cash_flows())
bank_bill_2 = inst.Bank_bill(face_value=100, maturity=.5)
bank_bill_2.set_ytm(.06)
bank_bill_2.set_cash_flows()
print(bank_bill_2.get_cash_flows())
yc_portfolio = inst.Portfolio()
yc_portfolio.add_bank_bill(bank_bill_1)
yc_portfolio.add_bank_bill(bank_bill_2)
# print(yc_portfolio.get_cash_flows())

# create a yield curve based on the bank bill portfolio
yc=yCurve.YieldCurve()
yc.set_constituent_portfolio(yc_portfolio)
yc.bootstrap()
print(yc.get_zero_curve())

# create a bond and add it to the portfolio
bond = inst.Bond(face_value=100, maturity=1, coupon=.06, frequency=2)
bond.set_ytm(.07)
bond.set_cash_flows()
print(bond.get_cash_flows())
bond2 = inst.Bond(face_value=100, maturity=2, coupon=.08, frequency=1)
bond2.set_ytm(.09)
bond2.set_cash_flows()
print(bond2.get_cash_flows())

yc_portfolio.add_bond(bond)
yc_portfolio.add_bond(bond2)
yc_portfolio.set_cash_flows()
print(yc_portfolio.get_cash_flows())

yc2=yCurve.YieldCurve()
yc2.set_constituent_portfolio(yc_portfolio)
yc2.bootstrap()
print(yc2.get_zero_curve())


[(0, -98.76543209876543), (0.25, 100)]
[(0, -97.08737864077669), (0.5, 100)]
([0, 0.25, 0.5], [1.0, 0.9876543209876543, 0.9708737864077669])
[(0, -99.05015286237719), (0.5, 3.0), (1, 103.0)]
[(0, -98.24088881407287), (1.0, 8.0), (2, 108.0)]
[(0, -99.05015286237719), (0.5, 3.0), (1, 103.0), (0, -98.24088881407287), (1.0, 8.0), (2, 108.0), (0, -98.76543209876543), (0.25, 100), (0, -97.08737864077669), (0.5, 100)]
([0, 0.25, 0.5, 1, 2], [1.0, 0.9876543209876543, 0.9708737864077669, 0.93337409226363, 0.8404990377404059])
